### Import dependencies

In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import datetime as dt
from sklearn.metrics import confusion_matrix, classification_report

### Merge datasets

In [2]:
# Read in data
cpi = pd.read_csv("resources/cpi_final.csv")
gdp = pd.read_csv("resources/gdp_final.csv")
gdp_pct = pd.read_csv("resources/gdp_pct_chg_final.csv")
houst = pd.read_csv("resources/housing_starts_final.csv")
opg = pd.read_csv("resources/output_gap_final.csv")
rec_dt = pd.read_csv("resources/recession_dates_final.csv")
unrate = pd.read_csv("resources/unemployment_rate_final.csv")
fed_funds = pd.read_csv("resources/fed_funds_final.csv")
yield10_2 = pd.read_csv("resources/10YT_minus_2YT_final.csv")
fred = pd.read_csv("resources/FRED_data.csv")

In [3]:
# Combine all data sets into one data frame
dfs = [cpi, gdp, gdp_pct, houst, opg, rec_dt, unrate, fed_funds, yield10_2, fred]
df = reduce(lambda left,right: pd.merge(left,right,on=['quarter'],how='outer'), dfs)

In [4]:
# Sort data frame by quarter
df = df.sort_values(by=['quarter'])

In [5]:
# Drop date columns
df = df.drop(columns=['date_x','date_y'])

# Rename target column
df = df.rename(columns={'target':'recession_actual'})

In [6]:
# Set index to quarter
df = df.set_index('quarter')

In [7]:
# Save 2019 Q1 & Q2
df_2019 = df.iloc[[-4,-3],:]
df_2019 = df_2019.drop(columns=['recession_actual'])
df_2019

,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,fed_funds_st_dev_rate,10YT_minus_2YT_avg,10YT_minus_2YT_percent_change_prev_quarter,real_disp_pers_inc,personal_consumption_exp_excl_food_energy,cpi_US_total,tot_public_debt_as_pct_of_gdp,gross_private_domestic_invest,M2_velocity,median_sls_price_houses_sold_US,nat_rate_of_unemp_long_term,personal_consumption_expenditures
quarter,,,,,,,,,,,,,,,,,,,,
2019Q1,253.311333,21098.827,3.9,1213.000000,0.848147,4.133333,2.401311,0.083088,0.004646,0.170000,-0.271429,4.5,1.6,1.644936,104.40334,3783.364,1.458,313000.0,4.577,14266.250
2019Q2,255.139333,21340.267,4.7,1255.666667,0.828815,3.500000,2.397813,-0.001457,0.024002,0.213333,0.254902,2.4,1.6,1.811376,103.20060,3749.471,1.457,322500.0,4.572,14511.176


In [8]:
# Drop rows with missing values
df = df.dropna()
df.tail()

,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,recession_actual,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,fed_funds_st_dev_rate,...,10YT_minus_2YT_percent_change_prev_quarter,real_disp_pers_inc,personal_consumption_exp_excl_food_energy,cpi_US_total,tot_public_debt_as_pct_of_gdp,gross_private_domestic_invest,M2_velocity,median_sls_price_houses_sold_US,nat_rate_of_unemp_long_term,personal_consumption_expenditures
quarter,,,,,,,,,,,,,,,,,,,,,
2018Q1,249.250333,20163.159,5.0,1320.666667,0.202456,0.0,4.333333,1.448966,0.204683,0.083902,...,-0.113861,6.9,1.8,2.214194,104.59493,3542.412,1.451,331800.0,4.597,13728.357
2018Q2,250.578667,20510.177,7.1,1259.666667,0.589182,0.0,3.833333,1.727176,0.192007,0.075492,...,-0.251397,2.7,2.0,2.711887,103.33928,3561.592,1.461,315600.0,4.592,13939.828
2018Q3,251.828667,20749.752,4.8,1233.000000,0.821959,0.0,3.866667,1.923492,0.113663,0.047184,...,-0.432836,3.3,2.0,2.640940,103.69309,3683.981,1.462,330900.0,4.587,14114.559
2018Q4,252.759000,20897.804,2.9,1185.000000,0.592021,0.0,3.566667,2.217097,0.152641,0.066218,...,-0.078947,2.8,1.9,2.203131,105.15026,3725.234,1.462,322800.0,4.582,14211.920
2019Q1,253.311333,21098.827,3.9,1213.000000,0.848147,0.0,4.133333,2.401311,0.083088,0.004646,...,-0.271429,4.5,1.6,1.644936,104.40334,3783.364,1.458,313000.0,4.577,14266.250


### Shift data with sliding window technique

In [9]:
df['recession_1q_out'] = df['recession_actual'].shift(-1)
df['recession_2q_out'] = df['recession_actual'].shift(-2)
df['recession_4q_out'] = df['recession_actual'].shift(-4)

In [10]:
# Create three datasets -- 1 for each model (recession 1Qtr out, 2Qtrs out, 4Qtrs out)
df_q1 = df.drop(columns=['recession_2q_out','recession_4q_out','recession_actual'])
df_q2 = df.drop(columns=['recession_4q_out','recession_1q_out','recession_actual'])
df_q4 = df.drop(columns=['recession_1q_out','recession_2q_out','recession_actual'])

In [11]:
# Delete missing values
df_q1 = df_q1.dropna()
df_q2 = df_q2.dropna()
df_q4 = df_q4.dropna()

In [12]:
# Define y variables
y1 = df_q1['recession_1q_out']
y2 = df_q2['recession_2q_out']
y3 = df_q4['recession_4q_out']

In [13]:
# Drop target
df_q1 = df_q1.drop(columns=['recession_1q_out'])
df_q2 = df_q2.drop(columns=['recession_2q_out'])
df_q4 = df_q4.drop(columns=['recession_4q_out'])

In [14]:
# Define X
X_q1 = df_q1
X_q2 = df_q2
X_q4 = df_q4

### Split and scale data

In [15]:
# Split data into training and testing (stratify=None, shuffle=False)
X1_train, X1_test, y1_train, y1_test=train_test_split(X_q1, y1, train_size=0.8, random_state=42, shuffle=False)
X2_train, X2_test, y2_train, y2_test=train_test_split(X_q2, y2, train_size=0.8, random_state=42, shuffle=False)
X3_train, X3_test, y3_train, y3_test=train_test_split(X_q4, y3, train_size=0.8, random_state=42, shuffle=False)

In [16]:
# Create scaler object
X1_scaler = StandardScaler().fit(X1_train)
X2_scaler = StandardScaler().fit(X2_train)
X3_scaler = StandardScaler().fit(X3_train)

# X full scaler
X1_full_scaler = StandardScaler().fit(X_q1)
X2_full_scaler = StandardScaler().fit(X_q2)
X3_full_scaler = StandardScaler().fit(X_q4)

In [17]:
# Scale training data
X1_train_scaled = X1_scaler.transform(X1_train)
X2_train_scaled = X2_scaler.transform(X2_train)
X3_train_scaled = X3_scaler.transform(X3_train)

# Scale testing data
X1_test_scaled = X1_scaler.transform(X1_test)
X2_test_scaled = X2_scaler.transform(X2_test)
X3_test_scaled = X3_scaler.transform(X3_test)

# Scale full X data (no splits)
X1_full_scaled = X1_full_scaler.transform(X_q1)
X2_full_scaled = X2_full_scaler.transform(X_q2)
X3_full_scaled = X3_full_scaler.transform(X_q4)

### Reshape data to fit LSTM format

In [18]:
# Method to reshape data
def reshape_data(obj):
    reshaped_obj = np.reshape(obj, (obj.shape[0], obj.shape[1], 1))
    return reshaped_obj

In [19]:
# Reshape training data
reshaped_X1_train_scaled = reshape_data(X1_train_scaled)
reshaped_X2_train_scaled = reshape_data(X2_train_scaled)
reshaped_X3_train_scaled = reshape_data(X3_train_scaled)

In [20]:
# Reshape testing data
reshaped_X1_test_scaled = reshape_data(X1_test_scaled)
reshaped_X2_test_scaled = reshape_data(X2_test_scaled)
reshaped_X3_test_scaled = reshape_data(X3_test_scaled)

In [21]:
# Reshape X_full
reshaped_X1_full = reshape_data(X1_full_scaled)
reshaped_X2_full = reshape_data(X2_full_scaled)
reshaped_X3_full = reshape_data(X3_full_scaled)

# Build Model

In [22]:
# Initialize model
model = Sequential()

In [23]:
# Add layers
model.add(LSTM(128, input_shape=(reshaped_X1_train_scaled.shape[1],1), return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())  # Normalize activation outputs

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(2, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [26]:
# Compile model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

## Train and predict on X1-Y1 data (recession 1 quarter out)

In [27]:
# Fit the model to the training data
model.fit(reshaped_X1_train_scaled, y1_train, validation_split=0.2, epochs=100, shuffle=False, verbose=2)

Train on 108 samples, validate on 28 samples
Epoch 1/100
108/108 - 17s - loss: 0.1393 - acc: 0.9444 - val_loss: 1.4172 - val_acc: 0.3214
Epoch 2/100
108/108 - 2s - loss: 0.1656 - acc: 0.9259 - val_loss: 1.0444 - val_acc: 0.4286
Epoch 3/100
108/108 - 2s - loss: 0.2049 - acc: 0.8981 - val_loss: 1.1493 - val_acc: 0.3929
Epoch 4/100
108/108 - 2s - loss: 0.2271 - acc: 0.8704 - val_loss: 1.3686 - val_acc: 0.2857
Epoch 5/100
108/108 - 2s - loss: 0.1922 - acc: 0.9352 - val_loss: 1.6555 - val_acc: 0.1429
Epoch 6/100
108/108 - 2s - loss: 0.1256 - acc: 0.9352 - val_loss: 1.4434 - val_acc: 0.1786
Epoch 7/100
108/108 - 2s - loss: 0.1330 - acc: 0.9352 - val_loss: 1.1400 - val_acc: 0.3214
Epoch 8/100
108/108 - 2s - loss: 0.1897 - acc: 0.9352 - val_loss: 0.9792 - val_acc: 0.4643
Epoch 9/100
108/108 - 2s - loss: 0.1070 - acc: 0.9722 - val_loss: 0.9147 - val_acc: 0.5357
Epoch 10/100
108/108 - 2s - loss: 0.1287 - acc: 0.9537 - val_loss: 0.8869 - val_acc: 0.5714
Epoch 11/100
108/108 - 2s - loss: 0.1173 - 

Epoch 90/100
108/108 - 2s - loss: 0.0637 - acc: 0.9630 - val_loss: 1.9688 - val_acc: 0.6786
Epoch 91/100
108/108 - 2s - loss: 0.0585 - acc: 0.9815 - val_loss: 2.0406 - val_acc: 0.7143
Epoch 92/100
108/108 - 2s - loss: 0.0538 - acc: 0.9722 - val_loss: 2.6530 - val_acc: 0.7500
Epoch 93/100
108/108 - 2s - loss: 0.0288 - acc: 0.9815 - val_loss: 2.8731 - val_acc: 0.7500
Epoch 94/100
108/108 - 2s - loss: 0.0269 - acc: 0.9907 - val_loss: 2.9271 - val_acc: 0.7500
Epoch 95/100
108/108 - 2s - loss: 0.0299 - acc: 0.9907 - val_loss: 2.8262 - val_acc: 0.7500
Epoch 96/100
108/108 - 2s - loss: 0.0189 - acc: 1.0000 - val_loss: 2.5323 - val_acc: 0.7500
Epoch 97/100
108/108 - 2s - loss: 0.0278 - acc: 0.9815 - val_loss: 2.3519 - val_acc: 0.7500
Epoch 98/100
108/108 - 2s - loss: 0.0162 - acc: 1.0000 - val_loss: 2.3568 - val_acc: 0.7500
Epoch 99/100
108/108 - 2s - loss: 0.0065 - acc: 1.0000 - val_loss: 2.4368 - val_acc: 0.7500
Epoch 100/100
108/108 - 2s - loss: 0.0174 - acc: 0.9907 - val_loss: 2.6155 - val

In [28]:
# Evaluate model using test data
model_loss1, model_accuracy1 = model.evaluate(reshaped_X1_test_scaled, y1_test, verbose=2)

34/34 - 0s - loss: 0.0096 - acc: 1.0000


In [29]:
# Make predictions using test data
predictions1_prob = model.predict_proba(reshaped_X1_test_scaled)
predictions1_class = model.predict_classes(reshaped_X1_test_scaled)

In [30]:
# Compare results
one_qtr_out = pd.DataFrame({"Predicted Probability of No Recession":predictions1_prob[:,0], "Predicted Probability of a Recession":predictions1_prob[:,1], "Actual":y1_test})
one_qtr_out.head()

,Predicted Probability of No Recession,Predicted Probability of a Recession,Actual
quarter,,,
2010Q3,0.978857,0.021143,0.0
2010Q4,0.981558,0.018442,0.0
2011Q1,0.999493,0.000507,0.0
2011Q2,0.999990,0.000010,0.0
2011Q3,0.999988,0.000012,0.0


#### Confusion Matrix on X1-Y1 data (recession 1 quarter out)

In [31]:
# Create confusion matrix on X1 model
con_mat = confusion_matrix(y1_test, predictions1_class)
print(con_mat)

[[34]]


In [32]:
# Score model
print(classification_report(y1_test, predictions1_class))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        34

    accuracy                           1.00        34
   macro avg       1.00      1.00      1.00        34
weighted avg       1.00      1.00      1.00        34



In [33]:
# Save model
name1 = f"unshuffled-1q-out-{dt.datetime.now()}"
model.save(f"models/{name1}.h5")

### Predict on 2019

In [34]:
# Scale 2019 data 
scaled_X1_2019 = X1_scaler.transform(df_2019)

# Reshape 2019 data
reshaped_X1_2019 = reshape_data(scaled_X1_2019)

# Predict on 2019
pred_X1_2019 = model.predict_proba(reshaped_X1_2019)
print(f"2019Q1 No Recession Probability: {pred_X1_2019[0][0]*100}")
print(f"2019Q1 Recession Probability: {pred_X1_2019[0][1]*100}")
print(f"2019Q2 No Recession Probability: {pred_X1_2019[1][0]*100}")
print(f"2019Q2 Recession Probability: {pred_X1_2019[1][1]*100}")

2019Q1 No Recession Probability: 99.9988317489624
2019Q1 Recession Probability: 0.0011696683941408992
2019Q2 No Recession Probability: 99.99895095825195
2019Q2 Recession Probability: 0.0010549343642196618


#### Predict on full X1

In [35]:
pred_X1_full = model.predict_classes(reshaped_X1_full)

# Preview results
X1_full_results = pd.DataFrame({"Predicted":pred_X1_full, "Actual":y1})
# X1_full_results.loc[X1_full_results["Actual"]==1]

# Export results for graphing
X1_full_results.to_csv(f"resources/predictions/X1_NS_VS20_{dt.datetime.now()}.csv")

## Train and predict on X2-Y2 data (recession 2 quarters out)

In [36]:
# Fit the model to the training data
model.fit(reshaped_X2_train_scaled, y2_train, validation_split=0.2, epochs=100, shuffle=False, verbose=2)

Train on 108 samples, validate on 27 samples
Epoch 1/100
108/108 - 2s - loss: 0.5225 - acc: 0.9259 - val_loss: 2.3169 - val_acc: 0.7407
Epoch 2/100
108/108 - 2s - loss: 0.2264 - acc: 0.9352 - val_loss: 2.0404 - val_acc: 0.7407
Epoch 3/100
108/108 - 2s - loss: 0.1287 - acc: 0.9537 - val_loss: 1.5759 - val_acc: 0.7407
Epoch 4/100
108/108 - 2s - loss: 0.1187 - acc: 0.9444 - val_loss: 1.2327 - val_acc: 0.7407
Epoch 5/100
108/108 - 2s - loss: 0.0717 - acc: 0.9537 - val_loss: 0.9981 - val_acc: 0.7407
Epoch 6/100
108/108 - 2s - loss: 0.1995 - acc: 0.9259 - val_loss: 0.8269 - val_acc: 0.7037
Epoch 7/100
108/108 - 2s - loss: 0.0736 - acc: 0.9630 - val_loss: 1.0899 - val_acc: 0.5556
Epoch 8/100
108/108 - 2s - loss: 0.1013 - acc: 0.9537 - val_loss: 1.2113 - val_acc: 0.5556
Epoch 9/100
108/108 - 2s - loss: 0.0776 - acc: 0.9815 - val_loss: 1.2328 - val_acc: 0.6296
Epoch 10/100
108/108 - 2s - loss: 0.0512 - acc: 1.0000 - val_loss: 1.2354 - val_acc: 0.5926
Epoch 11/100
108/108 - 2s - loss: 0.0577 - a

Epoch 90/100
108/108 - 2s - loss: 0.0239 - acc: 0.9907 - val_loss: 1.9363 - val_acc: 0.5556
Epoch 91/100
108/108 - 2s - loss: 0.0368 - acc: 0.9815 - val_loss: 1.7896 - val_acc: 0.5185
Epoch 92/100
108/108 - 2s - loss: 0.0321 - acc: 0.9907 - val_loss: 1.6632 - val_acc: 0.4815
Epoch 93/100
108/108 - 2s - loss: 0.0061 - acc: 1.0000 - val_loss: 1.7152 - val_acc: 0.5185
Epoch 94/100
108/108 - 2s - loss: 0.0042 - acc: 1.0000 - val_loss: 1.6728 - val_acc: 0.4815
Epoch 95/100
108/108 - 2s - loss: 0.0303 - acc: 0.9907 - val_loss: 1.7040 - val_acc: 0.5556
Epoch 96/100
108/108 - 2s - loss: 0.0035 - acc: 1.0000 - val_loss: 1.6816 - val_acc: 0.5185
Epoch 97/100
108/108 - 2s - loss: 0.0031 - acc: 1.0000 - val_loss: 1.5449 - val_acc: 0.5556
Epoch 98/100
108/108 - 2s - loss: 0.0204 - acc: 0.9907 - val_loss: 1.3606 - val_acc: 0.5185
Epoch 99/100
108/108 - 2s - loss: 0.0167 - acc: 0.9907 - val_loss: 1.1582 - val_acc: 0.5185
Epoch 100/100
108/108 - 2s - loss: 0.0157 - acc: 0.9907 - val_loss: 1.0415 - val

In [37]:
# Evaluate model using test data
model_loss2, model_accuracy2 = model.evaluate(reshaped_X2_test_scaled, y2_test, verbose=2)

34/34 - 0s - loss: 0.0532 - acc: 1.0000


In [38]:
# Make predictions using test data
predictions2_prob = model.predict_proba(reshaped_X2_test_scaled)
predictions2_class = model.predict_classes(reshaped_X2_test_scaled)

In [39]:
# Compare results
two_qtrs_out = pd.DataFrame({"Predicted Probability of No Recession":predictions2_prob[:,0], "Predicted Probability of a Recession":predictions2_prob[:,1], "Actual":y2_test})
two_qtrs_out.head()

,Predicted Probability of No Recession,Predicted Probability of a Recession,Actual
quarter,,,
2010Q2,0.999124,0.000876,0.0
2010Q3,0.999161,0.000839,0.0
2010Q4,0.999157,0.000843,0.0
2011Q1,0.999108,0.000892,0.0
2011Q2,0.991246,0.008754,0.0


#### Confusion Matrix on X2-Y2 data (recession 2 quarters out)

In [40]:
# Create confusion matrix on X2 model
con_mat = confusion_matrix(y2_test, predictions2_class)
print(con_mat)

[[34]]


In [41]:
# Score model
print(classification_report(y2_test, predictions2_class))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        34

    accuracy                           1.00        34
   macro avg       1.00      1.00      1.00        34
weighted avg       1.00      1.00      1.00        34



In [42]:
# Save model
name2 = f"unshuffled-2q-out-{dt.datetime.now()}"
model.save(f"models/{name2}.h5")

### Predict on 2019

In [43]:
# Scale 2019 data 
scaled_X2_2019 = X2_scaler.transform(df_2019)

# Reshape 2019 data
reshaped_X2_2019 = reshape_data(scaled_X2_2019)

# Predict on 2019
pred_X2_2019 = model.predict_proba(reshaped_X2_2019)
print(f"2019Q1 No Recession Probability: {pred_X2_2019[0][0]*100}")
print(f"2019Q1 Recession Probability: {pred_X2_2019[0][1]*100}")
print(f"2019Q2 No Recession Probability: {pred_X2_2019[1][0]*100}")
print(f"2019Q2 Recession Probability: {pred_X2_2019[1][1]*100}")

2019Q1 No Recession Probability: 72.01816439628601
2019Q1 Recession Probability: 27.98183262348175
2019Q2 No Recession Probability: 89.47288393974304
2019Q2 Recession Probability: 10.527120530605316


#### Predict on full X2

In [44]:
pred_X2_full = model.predict_classes(reshaped_X2_full)

# Preview results
X2_full_results = pd.DataFrame({"Predicted":pred_X2_full, "Actual":y2})
# X2_full_results.loc[X2_full_results["Actual"]==1]

# Export results for graphing
X2_full_results.to_csv(f"resources/predictions/X2_NS_VS20_{dt.datetime.now()}.csv")

### Train and predict on X3-Y3 data (recession 4 quarters out)

In [45]:
# Fit the model to the training data
model.fit(reshaped_X3_train_scaled, y3_train, validation_split=0.2, epochs=100, shuffle=False, verbose=2)

Train on 106 samples, validate on 27 samples
Epoch 1/100
106/106 - 2s - loss: 1.4917 - acc: 0.8396 - val_loss: 1.0848 - val_acc: 0.5926
Epoch 2/100
106/106 - 2s - loss: 0.6488 - acc: 0.8491 - val_loss: 1.0192 - val_acc: 0.7037
Epoch 3/100
106/106 - 2s - loss: 0.4657 - acc: 0.8679 - val_loss: 0.6691 - val_acc: 0.7778
Epoch 4/100
106/106 - 2s - loss: 0.3079 - acc: 0.8868 - val_loss: 0.7192 - val_acc: 0.8519
Epoch 5/100
106/106 - 2s - loss: 0.2174 - acc: 0.8962 - val_loss: 0.9689 - val_acc: 0.7407
Epoch 6/100
106/106 - 2s - loss: 0.1521 - acc: 0.9340 - val_loss: 1.1834 - val_acc: 0.6296
Epoch 7/100
106/106 - 2s - loss: 0.1920 - acc: 0.9057 - val_loss: 1.4789 - val_acc: 0.6667
Epoch 8/100
106/106 - 2s - loss: 0.1631 - acc: 0.9434 - val_loss: 1.3796 - val_acc: 0.5556
Epoch 9/100
106/106 - 2s - loss: 0.1565 - acc: 0.9245 - val_loss: 1.3144 - val_acc: 0.5185
Epoch 10/100
106/106 - 2s - loss: 0.1246 - acc: 0.9528 - val_loss: 1.3208 - val_acc: 0.4815
Epoch 11/100
106/106 - 2s - loss: 0.1145 - a

Epoch 90/100
106/106 - 1s - loss: 0.0151 - acc: 0.9906 - val_loss: 2.8885 - val_acc: 0.5556
Epoch 91/100
106/106 - 2s - loss: 0.0029 - acc: 1.0000 - val_loss: 2.8205 - val_acc: 0.5185
Epoch 92/100
106/106 - 2s - loss: 0.0088 - acc: 1.0000 - val_loss: 2.8509 - val_acc: 0.4815
Epoch 93/100
106/106 - 2s - loss: 0.0101 - acc: 1.0000 - val_loss: 3.0289 - val_acc: 0.4815
Epoch 94/100
106/106 - 2s - loss: 0.0038 - acc: 1.0000 - val_loss: 3.2717 - val_acc: 0.4074
Epoch 95/100
106/106 - 2s - loss: 0.0027 - acc: 1.0000 - val_loss: 3.4347 - val_acc: 0.4074
Epoch 96/100
106/106 - 2s - loss: 0.0061 - acc: 1.0000 - val_loss: 3.5212 - val_acc: 0.4074
Epoch 97/100
106/106 - 2s - loss: 0.0027 - acc: 1.0000 - val_loss: 3.5698 - val_acc: 0.4074
Epoch 98/100
106/106 - 2s - loss: 0.0051 - acc: 1.0000 - val_loss: 3.5970 - val_acc: 0.4074
Epoch 99/100
106/106 - 2s - loss: 0.0020 - acc: 1.0000 - val_loss: 3.6216 - val_acc: 0.4074
Epoch 100/100
106/106 - 2s - loss: 0.0021 - acc: 1.0000 - val_loss: 3.6031 - val

In [46]:
# Evaluate model using test data
model_loss3, model_accuracy3 = model.evaluate(reshaped_X3_test_scaled, y3_test, verbose=2)

34/34 - 0s - loss: 3.9641 - acc: 0.2059


In [47]:
# Make predictions using test data
predictions3_prob = model.predict_proba(reshaped_X3_test_scaled)
predictions3_class = model.predict_classes(reshaped_X3_test_scaled)

In [48]:
# Compare results
four_qtrs_out = pd.DataFrame({"Predicted Probability of No Recession":predictions3_prob[:,0], "Predicted Probability of a Recession":predictions3_prob[:,1], "Actual":y3_test})
four_qtrs_out.head()

,Predicted Probability of No Recession,Predicted Probability of a Recession,Actual
quarter,,,
2009Q4,0.025571,0.974429,0.0
2010Q1,0.016125,0.983875,0.0
2010Q2,0.002172,0.997828,0.0
2010Q3,0.000269,0.999731,0.0
2010Q4,0.000537,0.999463,0.0


#### Confusion Matrix on X3-Y3 data (recession 4 quarters out)

In [49]:
# Create confusion matrix on X3 model
con_mat = confusion_matrix(y3_test, predictions3_class)
print(con_mat)

[[ 7 27]
 [ 0  0]]


In [50]:
# Score model
print(classification_report(y3_test, predictions3_class))

              precision    recall  f1-score   support

         0.0       1.00      0.21      0.34        34
         1.0       0.00      0.00      0.00         0

    accuracy                           0.21        34
   macro avg       0.50      0.10      0.17        34
weighted avg       1.00      0.21      0.34        34



/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [51]:
# Save model
name3 = f"unshuffled-4q-out-{dt.datetime.now()}"
model.save(f"models/{name3}.h5")

### Predict on 2019

In [52]:
# Scale 2019 data 
scaled_X3_2019 = X3_scaler.transform(df_2019)

# Reshape 2019 data
reshaped_X3_2019 = reshape_data(scaled_X3_2019)

# Predict on 2019
pred_X3_2019 = model.predict_proba(reshaped_X3_2019)
print(f"2019Q1 No Recession Probability: {pred_X3_2019[0][0]*100}")
print(f"2019Q1 Recession Probability: {pred_X3_2019[0][1]*100}")
print(f"2019Q2 No Recession Probability: {pred_X3_2019[1][0]*100}")
print(f"2019Q2 Recession Probability: {pred_X3_2019[1][1]*100}")

2019Q1 No Recession Probability: 9.106820821762085
2019Q1 Recession Probability: 90.89317917823792
2019Q2 No Recession Probability: 20.08049786090851
2019Q2 Recession Probability: 79.91949915885925


#### Predict on full X3

In [53]:
pred_X3_full = model.predict_classes(reshaped_X3_full)

# Preview results
X3_full_results = pd.DataFrame({"Predicted":pred_X3_full, "Actual":y3})
# X3_full_results.loc[X3_full_results["Actual"]==1]

# Export results for graphing
X3_full_results.to_csv(f"resources/predictions/X3_NS_VS20_{dt.datetime.now()}.csv")